# Create Biomarker Supervisor Agent
In this notebook we create the Biomarker Supervisor Agent that will interact with all of the Biomarker sub-agents using the 'multi-agent-collaboration' feature in Bedrock Agents

# Prerequisites

1. Deploy the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account
2. Create all of the Biomarker sub-agents either through automatic deployment or development notebook method
3. Run through the notebook environment setup in agents_catalog/0-Notebook-environment

#### Ensure the latest version of boto3 is shown below

In [ ]:
!pip freeze | grep boto3

#### Load in environment variables to notebook

In [ ]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model needed
%store -r supervisor_agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [ ]:
%run $IMPORTS_PATH

# Agent Creation
In this section we create the supervisor agent

#### Define agent configuration below 

In [ ]:
agent_name = "multi-agent-biomarker"
agent_description = "Multi-agent collaboration for biomarker discovery"
agent_instruction = """You are a medical research assistant AI specialized in cancer biomarker analysis and discovery. 
Your primary task is to interpret user queries, use relevant agents for specific tasks, and provide consolidated 
medical insights based on the data. Use only the appropriate agents as required by the specific question. 
You can provide responses from a prior agent to the next agent in sequence. 

1. BiomarkerDatabaseAnalyst
- Executes SQL queries for biomarker data retrieval
- Manages patient biomarker database access
- Processes structured medical data
2. ClinicalEvidenceResearcher
- Synthesizes internal and external research evidence
- Provides context from existing medical literature
- Summarizes relevant clinical findings
3. MedicalImagingExpert
- Processes medical imaging scans
- Creates and manages imaging analysis jobs
- Interprets imaging results
4. Statistician
- Conducts survival analysis with biomarkers
- Performs statistical modeling
- Creates data visualizations

For each user query:
1. Query Analysis:
- Summarize understanding of the request
- Identify which specialized tools are needed
- Request any necessary clarifications
2. Execution Strategy:
- Coordinate relevant tools in logical sequence
- Pass information between tools as needed
- Track and validate results from each step
3. Response Format:
- Present findings from each utilized tool
- Provide integrated analysis across tools
- Explain technical concepts clearly
- Summarize research implications

Maintain scientific accuracy while ensuring explanations are accessible to users of varying expertise levels.
"""

#### Instantiate agent with the desired configuration

In [ ]:
agents = AgentsForAmazonBedrock()

supervisor_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    supervisor_agent_foundation_model,
    agent_collaboration='SUPERVISOR',
    code_interpretation=False,
    verbose=False,
)

supervisor_agent

#### Extract useful agent information

In [ ]:
supervisor_agent_id = supervisor_agent[0]
supervisor_agent_id

#### Define the sub-agents that the Supervisor Agent is to work with
#### Sub-agents can simply be added/removed from this list 
Note: Provide each agent's AliasID and Agent ID below

In [ ]:
# Retrieve the information from the Bedrock Agents console based on their names and fill in below

bedrock_agent_client = boto3.client('bedrock-agent', region)

# Biomarker-Database-Analyst
redshift_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='FILL IN',
    agentId='FILL IN'
)['agentAlias']['agentAliasArn']

# Clinical-evidence-researcher
research_evidence_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='FILL IN',
    agentId='FILL IN'
)['agentAlias']['agentAliasArn']

# Medical-imaging-expert
medical_imaging_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='FILL IN',
    agentId='FILL IN'
)['agentAlias']['agentAliasArn']

# Statistician
scientific_analysis_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='FILL IN',
    agentId='FILL IN'
)['agentAlias']['agentAliasArn']

redshift_agent_alias_arn, research_evidence_agent_alias_arn, medical_imaging_agent_alias_arn, scientific_analysis_agent_alias_arn

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': redshift_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in generating and executing SQL queries for a database containing medical biomarker information. Its primary task is to interpret user queries, generate and execute appropriate SQL queries.""",
        'sub_agent_association_name': 'BiomarkerDatabaseAnalyst',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': research_evidence_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in summarizing internal and external evidence related to cancer biomarkers. Its primary task is to interpret user queries, gather internal and external evidence, and provide relevant medical insights based on the results.""",
        'sub_agent_association_name': 'ClincialEvidenceResearcher',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': medical_imaging_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in processing medical imaging scans of patients. Its primary task is to create medical imaging jobs or provide relevant medical insights after jobs complete execution.""",
        'sub_agent_association_name': 'MedicalImagingExpert',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': scientific_analysis_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in in survival analysis with biomarkers. Its primary job is to interpret user queries, run scientific analysis tasks, and provide relevant medical insights with available visualization tools.""",
        'sub_agent_association_name': 'Statistician',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

In [ ]:
sub_agents_list

#### Associate sub-agents to supervisor, prepares supervisor agent

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = agents.associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn

#### Update SSM Parameter store so that Streamlit App uses new Biomarker Supervisor Agent

In [ ]:
ssm_client = boto3.client('ssm', region)

# Update AGENT_ID for Streamlit App
ssm_client.put_parameter(
    Name='/streamlitapp/env1/AGENT_ID',
    Value=supervisor_agent_id,
    Type='String',
    Overwrite=True
)

# Update AGENT_ALIAS_ID for Streamlit App
ssm_client.put_parameter(
    Name='/streamlitapp/env1/AGENT_ALIAS_ID',
    Value=supervisor_agent_alias_id,
    Type='String',
    Overwrite=True
)

## Ask supervisor agent different question types now that sub-agents are ready! ###

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())


# ---------------------------- Sample Question Bank --------------------------------------------
# Redshift Agent Questions
redshift_agent_query_1 = "How many patients are current smokers?"

# Research Evidence Agent Questions
research_evidence_agent_query_1 = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk?"

# Medical Imaging Agent Questions (must run in sequence)
medical_imaging_agent_query_1 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"
medical_imaging_agent_query_2 = "Can you higlight the elongation and sphericity of the tumor with these patients. Can you depict images of them?"

medical_imaging_agent_query_3 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values and then higlight the elongation and sphericity of the tumor with these patients?"

# Scientific Analysis Agent Questions
scientific_analysis_agent_query_1 = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy, Generate a bar chart of the top 5 gene biomarkers based on their p value and include their names in the x axis.?"


# -----------------------------------------------------------------------------------------


response = bedrock_agent_runtime_client.invoke_agent(
      inputText=redshift_agent_query_1, # Change value here to test different questions
      agentId=supervisor_agent_id,
      agentAliasId=supervisor_agent_alias_id, 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Supervisor Agent:\n{}".format(response))
print("====================")
print("Supervisor Agent processing query and collaborating with sub-agents to get answer")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

## Now the Biomarker Supervisor Agent is ready to assist you!